[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MobleyLab/drug-computing/blob/master/uci-pharmsci/assignments/MD/MD.ipynb)

# Molecular Dynamics (MD)

## Objective:
Perform some basic molecular dynamics (MD) simulations on a simple polymer model, perform some initial tests, and then after equilibrating the system, compute the self-diffusion coefficient for several different chain lengths. 

__Due date__: As assigned in class

## Overview:
One simple model of a polymer is just a chain of Lennard-Jones atoms. Here we will simulate such chains, interacting according to the potential (in dimensionless form): 

\begin{equation}
U^* = \sum \limits_{i<j \mathrm{,\  ij\  not\  bonded}} 4\left( r_{ij}^{-12} - r_{ij}^{-6}\right) + \sum \limits_{i<j\mathrm{,\  ij\  bonded}} \frac{k}{2} \left( r_{ij} - r_0\right)^2
\end{equation}

(Note that, as in the Energy Minimization assignment, we are using the dimensionless form, so that all of the constants are hidden in the units.)

Here, atoms have Lennard-Jones attractions and repulsions, and bonds between atoms along the polymer chain(s) are represented by simple harmonic springs. There are no torsional or angle potentials, and no electrostatic interactions. However, this simple model does share basic elements with the models we still use today for proteins and small molecules -- specifically, our classical MD models today begin with the potential above and add additional terms.
Simple systems like this polymer model have been thoroughly studied as models of polyatomic molecules, and as models of short polymers. It is relatively easy to derive or determine scaling laws for various physical properties as a function of polymer length in such systems. One such study, by Reis et al., [ Fluid Phase Equilibria 221: 25 (2004) ](https://doi.org/10.1016/j.fluid.2004.04.007) evaluated the self-diffusion coefficient for chains of different lengths. (The self-diffusion coefficient measures diffusive motion of something in a solution consisting of itself, for example the self-diffusion coefficient of water in water describes how mobile a water molecule is in pure water.)
Here, you will use some Python and Fortran libraries to set up some initial test simulations and make a plot relating to equilibration. Following that, you will compute the self-diffusion coefficient as directed below, making contact with the data of Reis et al. 

Most of the functions you will need have already been written for you and are provided here. Most of this assignment will involve using them to conduct a simulation. In addition to the paper mentioned, you will need `mdlib.f90` and `MD_functions.py`. As in the Energy Minimization assignment you did previously, you will need to compile `mdlib.f90` into a .so file suitable for use within Python.  

## Background/settings:
### Introduction of our variables:
Here, the potential energy will be as given above. Again, note that we are working in dimensionless form. 

We will simulate a system with a total of N monomers, some of which will be linked to form polymers. Each polymer will consist of M monomers, so that if $N_{poly}$ is the number of polymers, $N = M\times N_{poly}$. That is to say, we have $N_{poly}$ polymers each consisting of $M$ linked monomers in a chain, for a total of $N$ particles. 

As usual, our system will have a density, $\rho$, which is N/V. We will work with a particular temperature, $T$, and cutoff distance, $R_c$, beyond which Lennard-Jones interactions will not be included. Additionally, we need to specify a bond strength and equilibrium separation, $k$ and $r_0$, respectively. And we will take timesteps $\Delta t$ using the velocity Verlet integrator.

### Settings to use (unless otherwise noted)

Unless otherwise noted, here you should use the following settings:
* $k = 3000$ (spring constant)
* $r_0 = 1$ (preferred bond length)
* $N = 240$ (number of particles)
* $\rho = N/V = 0.8$ so that $L$, the box size, is $(N/\rho)^{1/3}$ 
* Use $L$ as the box size in your code
* $\Delta t = 0.001$ (timestep)
* $T = 1.0$ (temperature)
* $R_c = 2.5$ (call this Cut in your code)

Our use of the dimensionless form here includes setting all particle masses to 1. Because of this, forces and accelerations are the same thing. Additionally, units can be dropped, and the Boltzmann constant is equal to 1 in these units.


### What's provided

In this case, mdlib provides almost the same CalcEnergy and CalcEnergyForces routines you used in the previous assignment (for energy minimizations). Additionally, it provides a VVIntegrate function to actually use the integrator (Velocity Verlet) to take a timestep. You should look through the Fortran code to make sure you understand what is being done and see how it connects to what we covered in lecture.

The Python syntax for using VVintegrate looks like: 

`Pos, Vel, Accel, KEnergy, PEnergy = mdlib.vvintegrate( Pos, Vel, Accel, M, L, Cut, dt )` 

This takes a single timestep (covering time dt) and returns the position, velocity, acceleration, kinetic energy, and potential energy. 

Likewise, mdlib provides functions for calculating the potential energy, or the potential energy and forces, as:
`PEnergy = mdlib.calcenergy(Pos, M, L, Cut)`
and
`PE, Forces = mdlib.calcenergyforces(Pos, M, L, Cut, Forces)` 

By way of additional background/tips, Scott Shell has some very useful [simulation best practices](https://sites.engineering.ucsb.edu/~shell/che210d/Simulation_best_practices.pdf) tips which can help with thinking through how to code up and conduct effective simulations.

## Your assignment
All, or almost all, of the functions you will need to complete this assignment are described below. But before getting to the description, I want to explain the assignment. 

### Part A: Develop a simple molecular dynamics code and examine potential energy versus time for several values of M 
Edit the supplied code below (or MD.py if you prefer to work with the plain text; note that I have also provided MD_functions.py which is utilized by this notebook which provides only the functions you need and not a template for the code you need to write, since this is below) to develop a simple molecular dynamics code. Most of the functions you need are already provided (see documentation below, or in MD_functions.py). But, you do need to fill in the core of two functions:

* InitVelocities(N,T): Should take N, a number of particles and a target temperature and return a velocity array (‘Vel’) which is Nx3 with mean velocity corresponding to the correct average temperature. You may wish to do this by assigning random velocities and rescaling (see below).

* RescaleVelocities(Vel, T): Re-center the velocities to zero net momentum to remove any overall translational motion (i.e., subtract off the average velocity from all particles) and then re-scale the velocities to maintain the correct temperature. This can be done by noting that the average kinetic energy of the system is related in a simple way to the effective temperature: 

\begin{equation}
\frac{1}{2}\sum \limits_i m_i v_i^2 = \frac{3}{2} N k_B T
\end{equation}

The left-hand term is the kinetic energy, and here can be simplified by noting all of the masses in the system are defined to be 1. The right hand term involves the Boltzmann constant, the number of particles in the system, and the instantaneous temperature.

So, you can compute the effective temperature of your system, and translate this into a scaling factor which you can use to multiply (scale) all velocities in the system to ensure you get the correct average temperature (see https://www.compchems.com/thermostats-in-molecular-dynamics/#velocity-rescaling). **Specifically, following the page above, compute a (scalar) constant $\lambda$ by which you will multiply all of the velocities to ensure that the effective temperature is at the correct target value after rescaling.** To do this calculation you will need to compute the kinetic energy, which involves the sum above. 
    
Remove translational motion, rescale the velocities, and have your function return the  updated velocity array.
Once the above two functions are written, finishing write a simple MD code using the available functions to:
* Initially place atoms on a cubic lattice with the correct box size
* Energy-minimize the initial configuration using the conjugate-gradient energy minimizer; this will help ensure the simulation doesn’t “explode” (a highly technical term meaning “crash”) when you begin MD
* Assign initial velocities and compute forces (accelerations)
* Use the velocity Verlet integrator to perform a molecular dynamics run. Rescale atomic velocities every **RescaleFreq** integration steps to achieve the target temperature T. ( You can test whether you should rescale the velocities using the modulo (remainder) operator, for example $i % RescaleFreq == 0$)
   * You might want to use RescaleFreq = 100 (for extra credit, you can try several values of RescaleFreq and explain the differences in fluctuations in the potential energy versus time that you see)

Use the settings given above for $N$, $\rho$, $T$, the timestep, and the cutoff distance. 

Perform simulations for $M = 2, 4, 6, 8, 12,$ and $16$ and store the total energies (kinetic plus potential) versus time out to 2,000 timesteps. (Remember, $M$ controls the number of particles per polymer; you are keeping the same total number of particles in the system and changing the size of the polymers). On a single graph, plot the potential energy versus time for each of these cases (each in a different color). Turn in this graph.  

Note also you can visualize, if desired, using the Python module for writing to PDB files which you saw in the Energy Minimization exercise. 


### Part B: Extend your code to compute the self-diffusion coefficient as a function of chain length

Modify your MD code from above to perform a series of steps that will allow you to compute the self-diffusion coefficient as a function of chain length and determine how diffusion of polymers depends on the size of the polymer. To compute the self-diffusion coefficient, you will simply need to monitor the motion of each polymer in time. 

Here, you will first perform two equilibrations at constant temperature using velocity rescaling . The first will allow the system to reach the desired temperature and forget about its initial configuration (remember, it was started on a lattice). The second will allow you to compute the average total energy (kinetic plus potential) of the system. Then, you will fix the total energy at this value and perform a production simulation. 

Here’s what you should do:
* Following initial preparation (like above), first perform equilibration for NStepsEquil1 using velocity rescaling to target temperature T every RescaleFreq steps, using whatever value of RescaleFreq you used previously (NOT every step!)
* Perform a second equilibration for `NStepsEquil2` timesteps using velocity rescaling at the same frequency, storing energies (kinetic and potential, separately) while you do this. 
* Compute the average total energy (kinetic plus potential) over this second equilibration and rescale the velocities to start the final phase with the correct total energy. In other words, the total energy at the end of equilibration will be slightly above or below the average; you should find the kinetic energy you need to have to get the correct total energy, and rescale the velocities to get this kinetic energy. After this you will be doing no more velocity rescaling. (Hint: You can do this final rescaling easily by computing a scaling factor, and you will probably not be using the rescaling code you use to maintain the temperature during equilibration.)
* Copy the initial positions of the particles into a reference array for computing the mean squared displacement, for example using `Pos0 = Pos.copy()`
* Perform a production run for `NStepsProd` integration steps with constant energy (NVE) rather than velocity rescaling. Periodically record the time and mean squared displacement of the atoms from their initial positions. (You will need to write a small bit of code to compute mean squared displacements, but it shouldn’t take more than a couple of lines; you may send it to me to check if you are concerned about it). The mean squared displacement is given by 
\begin{equation}
\left<\left| \mathbf{r}-\mathbf{r_0} \right|^2 \right>
\end{equation}

  where the $\mathbf{r}$'s are the current and initial positions of the object in question so the mean squared   
  displacement measures the square of the distance traveled for the object. 

* Compute the self-diffusion coefficient, $D$. The mean squared displacement relates to the self-diffusion coefficient, $D$, in this way:
\begin{equation}
\left< \left| \mathbf{r} - \mathbf{r_0}\right| \right>^2 = 6 D t
\end{equation}

Here $D$ is the self-diffusion coefficient and t is the elapsed time. That is, the expected squared distance traveled (mean squared displacement) grows linearly with the elapsed time.

For settings, use NStepsEquil1 = 10,000 = NStepsEquil2 and NStepsProd = 100,000. (Note: You should probably do a “dry run” first with shorter simulations to ensure everything is working, as 100,000 steps might take an hour or more to run).

* Perform these runs for $M = 2, 4, 6, 8, 12,$ and $16$, storing results for each. 
* Plot the mean-squared displacement versus time for each M on the same graph. 
* Compute the diffusion coefficient for each $M$ from the slope of each graph and plot these fits on the same graph. You can do a linear least-squares fit easily in Numpy. 

`Slope, Intercept = np.polyfit( xvals, yvals, 1)`

Plot the diffusion coefficient versus $M$ and try and see if it follows any obvious scaling law. It should decrease with increasing $M$, but with what power? (You may want to refer to the Reis et al. paper). 

### What to turn in:
* Your plot of the potential energy versus time for each $M$ in Part A
* Mean-squared displacement versus time, and fit, for each $M$ in Part B, all on one plot
* The diffusion coefficient versus $M$ in Part B
* Your code for at least Part B
* Any comments you have - do you think you got it right? Why or why not? What was confusing/helpful? What would you do if you had more time? 
* Clearly label axes and curves on your plots!

You can send your comments/discussion as an e-mail, and the rest of the items as attachments.

### What’s provided for you: 
In this case, most of what you need is provided in the importable module `MD_functions.py` (which you can view with your favorite text editor, like `vi` or Atom), except for the functions for initial velocities and velocity rescaling -- in those cases, the shells are present below and you need to write the core (which will be very brief!).  **However, you will also need to insert the code for the `ConjugateGradient` function in `MD_functions.py`**. This is similar to (but not identical to) the ConjugateGradient code from the energy minimization assignment -- specifically, now our particles are joined into polymers, meaning that the function needs slightly different arguments to pass to energy and force evaluation routines. 

From the Fortran library `mdlib` (which you will compile as usual via `f2py3 -c -m mdlib mdlib.f90` or similar), the only new function you need is Velocity Verlet. In `MD_functions.py`, the following tools are available (this shows their documentation, not the details of the code, but you should only need to read the documentation in order to be able to use them. NOTE: No modification of these functions is needed; you only need to use them. You will only need to write `InitVelocities` and `RescaleVelocities` as described above, plus provide your previous code for `ConjugateGradient`: 


## Installing Packages

***If you are running this on Google Colab, please add the installation blocks from the [getting started notebook](https://github.com/MobleyLab/drug-computing/blob/master/uci-pharmsci/Getting_Started.ipynb) or [condacolab](https://github.com/aakankschit/drug-computing/blob/master/uci-pharmsci/Getting_Started_condacolab.ipynb) here and then execute the code below***


Help on module MD:

NAME

    MD - #MD exercise template for PharmSci 175/275


FUNCTIONS

FUNCTIONS
    
    ConjugateGradient(Pos, dx, EFracTolLS, EFracTolCG, M, L, Cut)
        Performs a conjugate gradient search.
        Input:
            Pos: starting positions, (N,3) array
            dx: initial step amount
            EFracTolLS: fractional energy tolerance for line search
            EFracTolCG: fractional energy tolerance for conjugate gradient
            M: Monomers per polymer
            L: Box size
            Cut: Cutoff
        Output:
            PEnergy: value of potential energy at minimum
            Pos: minimum energy (N,3) position array

    InitPositions(N, L)
        Returns an array of initial positions of each atom,
        placed on a cubic lattice for convenience.
        Input:
            N: number of atoms
            L: box length
        Output:
            Pos: (N,3) array of positions

    InitVelocities(N, T)
        Returns an initial random velocity set.
        Input:
            N: number of atoms
            T: target temperature
        Output:
            Vel: (N,3) array of atomic velocities

        InstTemp(Vel)
        Returns the instantaneous temperature.
        Input:
            Vel: (N,3) array of atomic velocities
        Output:
            Tinst: float
    
    InstTemp(Vel):
       Returns the instantaneous temperature.
       Input:
           Vel: (N,3) array of atomic velocities
       Output:
           Tinst: float

    RescaleVelocities(Vel, T)
        Rescales velocities in the system to the target temperature.
        Input:
            Vel: (N,3) array of atomic velocities
            T: target temperature
        Output:
            Vel: same as above 
            
    LineSearch(Pos, Dir, dx, EFracTol, M, L, Cut, Accel=1.5, MaxInc=10.0,  
               MaxIter=10000)
        Performs a line search along direction Dir.
        Input:
            Pos: starting positions, (N,3) array
            Dir: (N,3) array of gradient direction
            dx: initial step amount
            EFracTol: fractional energy tolerance
            M: Monomers per polymer
            L: Box size
            Cut: Cutoff
            Accel: acceleration factor
            MaxInc: the maximum increase in energy for bracketing
            MaxIter: maximum number of iteration steps
        Output:
            PEnergy: value of potential energy at minimum along Dir
            Pos: minimum energy (N,3) position array along Dir



## Here, you should actually write your functions:

In [ ]:
import mdlib
from MD_functions import *

def InitVelocities(N, T):
    """Returns an initial random velocity set.
Input:
    N: number of atoms
    T: target temperature
Output:
    Vel: (N,3) array of atomic velocities
"""
    #WRITE THIS CODE
    #THEN RETURN THE NEW VELOCITIES
    return Vel


def RescaleVelocities(Vel, T):
    """Rescales velocities in the system to the target temperature.
Input:
    Vel: (N,3) numpy array of atomic velocities
    T: target temperature
Output:
    Vel: same as above
"""
    #WRITE THIS CODE
    #recenter to zero net momentum (assuming all masses same)
    #find the total kinetic energy
    #find velocity scale factor from ratios of kinetic energy
    #Update velocities

    #NOW RETURN THE NEW VELOCITIES
    return Vel

## Now use your functions, coupled with those provided, to code up your assignment:

In [ ]:
#PART A:

#Define box size and other settings
k=3000 
r0=1 
N=240 
rho=0.8 #Solve to find L
#Set L
dt=0.001 
T=1.0 
Cut=2.5
RescaleFreq = 100 #See note above - may want to try several values

#Define your M value(s)

#Initially place atoms on a cubic lattice 
#Energy-minimize the initial configuration using the conjugate-gradient energy minimizer
#Assign initial velocities and compute forces (accelerations)
#Use the velocity Verlet integrator to perform a molecular dynamics run, rescaling velocities when appropriate

In [1]:
#PART B:
#Additionally:
NStepsEquil1 = 10000
NStepsEquil2 = 10000
NStepsProd = 100000

#Set up as in A
#Equilibrate for NStepsEquil1 with velocity rescaling every RescaleFreq steps, discarding energies
#Equilibrate for NStepsEquil2 with velocity rescaling, storing energies
#Stop and average the toal energy. Rescale the velocities so the current (end-of-equilibration) total energy matches the average total energy.
#Store the particle positions so you can later compute the mean squared displacement
#Run for NStepsProd at constant energy (NVE) recording time and mean squared displacement periodically

#Compute diffusion coefficient for each M using a fit to the mean squared displacement
#Plot mean squared displacement for each M
#Plot diffusion coefficient as a function of M

## Follow-up: 
The thermostat used here is not in general recommended, for reasons we will discuss in class. To understand one such reason, please read the “flying ice cube” paper referenced here: J Comp Chem 19:726-740 (1998) <http://onlinelibrary.wiley.com/doi/10.1002/(SICI)1096-987X(199805)19:7%3C726::AID-JCC4%3E3.0.CO;2-S/full>